# BayesCap SRGAN Experiment Suite
This notebook mirrors the new `scripts/bayescap_pipeline.py` CLI while keeping a linear, reproducible workflow for running every experiment combination (ImageNet vs. DIV2K checkpoints, parameter sweeps, and qualitative visualization) on the four benchmark super-resolution datasets.

## 1. Environment & Imports
Import the shared pipeline helpers plus the network definitions so every subsequent section (dataset setup, baseline evaluation, DIV2K training, parameter sweeps, qualitative viz) can call into the same code paths as the CLI.

In [1]:
import json
import sys
import random
from pathlib import Path

import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

project_root = Path.cwd().parent
scripts_dir = project_root / 'scripts'
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

from bayescap_pipeline import (
    organize_sr_benchmarks,
    evaluate_sr_metrics,
    run_parameter_sweep,
    compare_experiments,
    load_experiment_registry,
    download_div2k,
    extract_div2k,
    build_div2k_loaders,
    pretrain_srgan_on_div2k,
    finetune_bayescap_on_div2k,
    load_model_pair,
)
from ds import ImgDset
from networks_SRGAN import Generator, BayesCap
from utils import img_ssim

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

data_root = project_root / 'data'
benchmark_root = data_root / 'SR' / 'val'
sr_factor = 4
baseline_generator_ckpt = project_root / 'ckpt' / 'srgan-ImageNet-bc347d67.pth'
baseline_bayescap_ckpt = project_root / 'ckpt' / 'BayesCap_SRGAN_best.pth'

/data/oe23/BayesCap/src/utils.py:506: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:78.)
  dtype=torch.cuda.FloatTensor(),


## 2. Benchmark Dataset Organization
Organize Set5/Set14/BSD100/Urban100 HR images into `data/SR/val/<dataset>/original` so loaders and evaluation code can enumerate them consistently. Skip gracefully if the structure already exists.

> **Scripted workflow**  The heavy-lifting utilities now live in `../scripts/bayescap_pipeline.py`.  You can call the same routines from this notebook (imported below) or invoke them via `python ../scripts/bayescap_pipeline.py --help` for a pure command-line workflow.

In [2]:
datasets = ['Set5', 'Set14', 'BSD100', 'Urban100']
organize_sr_benchmarks(data_root, datasets, sr_factor=sr_factor)
val_root = benchmark_root

[Set5] copied 0 HR images to /data/oe23/BayesCap/data/SR/val/Set5/original
  (nothing to do; files already organized)
[Set14] copied 0 HR images to /data/oe23/BayesCap/data/SR/val/Set14/original
  (nothing to do; files already organized)
[BSD100] copied 0 HR images to /data/oe23/BayesCap/data/SR/val/BSD100/original
  (nothing to do; files already organized)
[Urban100] copied 0 HR images to /data/oe23/BayesCap/data/SR/val/Urban100/original
  (nothing to do; files already organized)
Done organizing SR benchmark datasets.


In [3]:
val_dataset_name = 'Set5'
val_dset = ImgDset(
    dataroot=str(benchmark_root / val_dataset_name / 'original'),
    image_size=(256, 256),
    upscale_factor=sr_factor,
    mode='val',
)
val_loader = DataLoader(
    val_dset,
    batch_size=1,
    shuffle=False,
    pin_memory=torch.cuda.is_available(),
)
print(f"Loaded {len(val_dset)} validation crops from {val_dataset_name}")

Loaded 5 validation crops from Set5


### Validation loader for qualitative inspection
A lightweight loader on Set5 powers the final visualization cell so we can peek at SR outputs, uncertainty maps, and reconstruction errors.

## 3. Baseline (ImageNet-pretrained) Checkpoints
Load the released SRGAN generator and BayesCap head so we always have a reference point before adapting to DIV2K.

In [4]:
NetG, NetC = load_model_pair(baseline_generator_ckpt, baseline_bayescap_ckpt, device)
netg_params = sum(p.numel() for p in NetG.parameters())
netc_params = sum(p.numel() for p in NetC.parameters())
print(f"Generator params: {netg_params:,}")
print(f"BayesCap params: {netc_params:,}")

Generator params: 1,547,350
BayesCap params: 2,589,658


In [5]:
benchmark_datasets = ['Set5', 'Set14', 'BSD100', 'Urban100']
baseline_metrics_df = evaluate_sr_metrics(
    NetG,
    NetC,
    benchmark_datasets,
    dataset_root=benchmark_root,
    image_size=(256, 256),
    upscale_factor=sr_factor,
    batch_size=1,
    num_bins=30,
    device=str(device),
    dtype=dtype,
)
baseline_metrics_df

,PSNR,SSIM,UCE,C.Coeff,Images
Dataset,,,,,
BSD100,24.286604,0.634499,0.066838,0.337899,100
Set14,23.612141,0.663415,0.072535,0.274057,14
Set5,26.784134,0.794881,0.068288,0.347054,5
Urban100,21.594371,0.633385,15.270981,0.249663,100


## 5. Experiment Registry (ImageNet vs. DIV2K)
Use the JSON registry in `../scripts/experiment_registry.example.json` (copy/edit as needed) to enumerate every generator/BayesCap checkpoint pair you want to evaluate. The helper below loops through each entry, evaluates it on all four datasets, and produces a single comparison table.

In [6]:
experiment_registry_path = project_root / 'scripts' / 'experiment_registry.example.json'
experiments = load_experiment_registry(experiment_registry_path)
comparison_df = compare_experiments(
    experiments,
    benchmark_root,
    ['Set5', 'Set14', 'BSD100', 'Urban100'],
    image_size=(256, 256),
    upscale_factor=sr_factor,
    batch_size=1,
    num_bins=30,
    device=device,
    dtype=dtype,
)
comparison_df

PSNR      SSIM           UCE   C.Coeff  \
Experiment          Dataset                                                 
div2kG_imagenetC    BSD100    23.367758  0.480238      0.007322  0.349033   
                    Set14     21.127391  0.426991      0.010483  0.356087   
                    Set5      22.468839  0.467435      0.016124  0.393581   
                    Urban100  19.735719  0.386285      0.014973  0.291797   
div2k_pretrained    BSD100    23.367758  0.480238      0.093616  0.065566   
                    Set14     21.127391  0.426991      0.062091       NaN   
                    Set5      22.468839  0.467435      0.061113  0.133503   
                    Urban100  19.735719  0.386285      5.814424       NaN   
imagenetG_div2kC    BSD100    24.286604  0.634499      0.367766       NaN   
                    Set14     23.612141  0.663415      0.088027       NaN   
                    Set5      26.784134  0.794881      0.105315  0.019175   
                    Urban100  21.594371  0.633385  13519.808522       NaN   
imagenet_pretrained BSD100    24.286604  0.634499      0.066838  0.337899   
                    Set14     23.612141  0.663415      0.072535  0.274057   
                    Set5      26.784134  0.794881      0.068288  0.347054   
                    Urban100  21.594371  0.633385     15.270981  0.249663   

                              Images                          Description  
Experiment          Dataset                                                
div2kG_imagenetC    BSD100       100  DIV2K generator + ImageNet BayesCap  
                    Set14         14  DIV2K generator + ImageNet BayesCap  
                    Set5           5  DIV2K generator + ImageNet BayesCap  
                    Urban100     100  DIV2K generator + ImageNet BayesCap  
div2k_pretrained    BSD100       100             DIV2K fine-tuned weights  
                    Set14         14             DIV2K fine-tuned weights  
                    Set5           5             DIV2K fine-tuned weights  
                    Urban100     100             DIV2K fine-tuned weights  
imagenetG_div2kC    BSD100       100  ImageNet generator + DIV2K BayesCap  
                    Set14         14  ImageNet generator + DIV2K BayesCap  
                    Set5           5  ImageNet generator + DIV2K BayesCap  
                    Urban100     100  ImageNet generator + DIV2K BayesCap  
imagenet_pretrained BSD100       100             Original release weights  
                    Set14         14             Original release weights  
                    Set5           5             Original release weights  
                    Urban100     100             Original release weights

## 6. Parameter sweep across datasets and scoring knobs
Explore sensitivity to crop size, batch size, dataset grouping, and UCE binning. Results are concatenated into a single DataFrame and the manifest is logged to `data/SR/val/bayescap_sweep_manifest.json`. Grayscale benchmarks (e.g., Set14) are now auto-expanded to RGB during collation so mixed-channel batches no longer crash the sweep.

In [7]:
sweep_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sweep_dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
dataset_options = [
    ['Set5', 'Set14', 'BSD100'],
    ['Set5', 'Set14', 'BSD100', 'Urban100']
]
image_sizes = [(128, 128), (256, 256), (320, 320)]
batch_sizes = [1, 2]
num_bins = [15, 20, 30, 40]
manifest_path = benchmark_root / 'bayescap_sweep_manifest.json'

sweep_df = run_parameter_sweep(
    NetG,
    NetC,
    benchmark_root,
    dataset_options,
    image_sizes,
    batch_sizes,
    num_bins,
    upscale_factor=sr_factor,
    device=sweep_device,
    dtype=sweep_dtype,
    manifest_path=manifest_path,
)
sweep_df

[cfg_01] datasets=['Set5', 'Set14', 'BSD100'], image_size=(128, 128), batch=1, num_bins=15
[cfg_02] datasets=['Set5', 'Set14', 'BSD100', 'Urban100'], image_size=(128, 128), batch=1, num_bins=15
[cfg_02] datasets=['Set5', 'Set14', 'BSD100', 'Urban100'], image_size=(128, 128), batch=1, num_bins=15
[cfg_03] datasets=['Set5', 'Set14', 'BSD100'], image_size=(128, 128), batch=1, num_bins=20
[cfg_03] datasets=['Set5', 'Set14', 'BSD100'], image_size=(128, 128), batch=1, num_bins=20
[cfg_04] datasets=['Set5', 'Set14', 'BSD100', 'Urban100'], image_size=(128, 128), batch=1, num_bins=20
[cfg_04] datasets=['Set5', 'Set14', 'BSD100', 'Urban100'], image_size=(128, 128), batch=1, num_bins=20
[cfg_05] datasets=['Set5', 'Set14', 'BSD100'], image_size=(128, 128), batch=1, num_bins=30
[cfg_05] datasets=['Set5', 'Set14', 'BSD100'], image_size=(128, 128), batch=1, num_bins=30
[cfg_06] datasets=['Set5', 'Set14', 'BSD100', 'Urban100'], image_size=(128, 128), batch=1, num_bins=30
[cfg_06] datasets=['Set5', 'Se

PSNR      SSIM       UCE   C.Coeff  Images   ImageSize  \
Config Dataset                                                                 
cfg_01 BSD100    23.509387  0.612648  0.076594  0.284839     100  (128, 128)   
       Set14     22.376287  0.626960  0.084797  0.255654      14  (128, 128)   
       Set5      24.194658  0.740373  0.097095  0.200861       5  (128, 128)   
cfg_02 BSD100    23.509387  0.612648  0.076594  0.284839     100  (128, 128)   
       Set14     22.376287  0.626960  0.084797  0.255654      14  (128, 128)   
...                    ...       ...       ...       ...     ...         ...   
cfg_47 Set5      28.024978  0.815751  0.060392  0.362039       5  (320, 320)   
cfg_48 BSD100    23.982155  0.623020  0.063944  0.333563     100  (320, 320)   
       Set14     24.242203  0.681729  0.068491  0.271168      14  (320, 320)   
       Set5      28.024978  0.815751  0.060392  0.362039       5  (320, 320)   
       Urban100  21.892282  0.652716  1.226360  0.271906     100  (320, 320)   

                 BatchSize  NumBins                       Datasets  
Config Dataset                                                      
cfg_01 BSD100            1       15            Set5, Set14, BSD100  
       Set14             1       15            Set5, Set14, BSD100  
       Set5              1       15            Set5, Set14, BSD100  
cfg_02 BSD100            1       15  Set5, Set14, BSD100, Urban100  
       Set14             1       15  Set5, Set14, BSD100, Urban100  
...                    ...      ...                            ...  
cfg_47 Set5              2       40            Set5, Set14, BSD100  
cfg_48 BSD100            2       40  Set5, Set14, BSD100, Urban100  
       Set14             2       40  Set5, Set14, BSD100, Urban100  
       Set5              2       40  Set5, Set14, BSD100, Urban100  
       Urban100          2       40  Set5, Set14, BSD100, Urban100  

[168 rows x 9 columns]

In [9]:
sweep_df.to_csv(project_root / 'results' / 'bayescap_sr_parameter_sweep_results.csv', index=False)

## 7. DIV2K pretraining & fine-tuning workflow
Download DIV2K HR images, build loaders, optionally fine-tune the SRGAN generator, and (optionally) adapt BayesCap using the newly trained backbone. Toggle the boolean flags when you're ready to launch the heavier jobs; the training helpers now expose `tqdm` progress bars so you can track batches directly in the notebook.

### 7.1 Download & extract DIV2K

In [10]:
div2k_url = 'http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip'
div2k_zip_path = download_div2k(data_root, div2k_url)
div2k_hr_dir = extract_div2k(div2k_zip_path)
print(f'DIV2K archive: {div2k_zip_path}')
print(f'DIV2K HR directory: {div2k_hr_dir}')

[skip] /data/oe23/BayesCap/data/DIV2K/DIV2K_train_HR.zip already present
[skip] /data/oe23/BayesCap/data/DIV2K/DIV2K_train_HR already extracted
DIV2K archive: /data/oe23/BayesCap/data/DIV2K/DIV2K_train_HR.zip
DIV2K HR directory: /data/oe23/BayesCap/data/DIV2K/DIV2K_train_HR


### 7.2 Build DIV2K loaders

In [11]:
div2k_train_loader, div2k_val_loader = build_div2k_loaders(
    div2k_hr_dir,
    sr_factor=sr_factor,
    train_crop=(128, 128),
    val_crop=(256, 256),
    train_batch=8,
    val_batch=4,
    num_workers=4,
)
print(f"DIV2K train batches: {len(div2k_train_loader)}, val batches: {len(div2k_val_loader)}")

DIV2K train batches: 100, val batches: 200


### 7.3 Optional SRGAN fine-tuning on DIV2K

In [13]:
div2k_generator_ckpt = project_root / 'ckpt' / 'srgan_DIV2K.pth'
run_div2k_training = True  # Set True to launch full DIV2K training

if run_div2k_training:
    NetG_div2k = Generator()
    pretrain_srgan_on_div2k(
        NetG_div2k,
        div2k_train_loader,
        div2k_val_loader,
        epochs=100,
        lr=5e-5,
        ckpt_path=div2k_generator_ckpt,
        device=device,
    )
else:
    print("Skipping DIV2K generator training (run_div2k_training=False)")

[DIV2K][Epoch 1/100] train:   0%|          | 0/100 [00:00<?, ?batch/s]

[DIV2K][Epoch 1/100] train loss=0.4157, val PSNR=13.03 dB
  -> saved new best checkpoint to /data/oe23/BayesCap/ckpt/srgan_DIV2K.pth


[DIV2K][Epoch 2/100] train loss=0.1755, val PSNR=15.29 dB
  -> saved new best checkpoint to /data/oe23/BayesCap/ckpt/srgan_DIV2K.pth


[DIV2K][Epoch 3/100] train loss=0.1397, val PSNR=17.02 dB
  -> saved new best checkpoint to /data/oe23/BayesCap/ckpt/srgan_DIV2K.pth


[DIV2K][Epoch 4/100] train loss=0.1215, val PSNR=17.98 dB
  -> saved new best checkpoint to /data/oe23/BayesCap/ckpt/srgan_DIV2K.pth


...

[DIV2K][Epoch 96/100] train loss=0.0522, val PSNR=24.20 dB
  -> saved new best checkpoint to /data/oe23/BayesCap/ckpt/srgan_DIV2K.pth


[DIV2K][Epoch 97/100] train loss=0.0523, val PSNR=24.19 dB


[DIV2K][Epoch 98/100] train loss=0.0522, val PSNR=24.20 dB
  -> saved new best checkpoint to /data/oe23/BayesCap/ckpt/srgan_DIV2K.pth


[DIV2K][Epoch 99/100] train loss=0.0523, val PSNR=24.19 dB


[DIV2K][Epoch 100/100] train loss=0.0522, val PSNR=24.20 dB


### 7.4 Optional BayesCap adaptation on DIV2K

In [14]:
div2k_bayescap_ckpt = project_root / 'ckpt' / 'BayesCap_SRGAN_DIV2K.pth'
run_div2k_bayescap = True  # Set True to adapt BayesCap with DIV2K checkpoints

if run_div2k_bayescap:
    if not div2k_generator_ckpt.exists():
        raise FileNotFoundError(f"DIV2K generator checkpoint missing at {div2k_generator_ckpt}")
    finetune_bayescap_on_div2k(
        div2k_generator_ckpt,
        div2k_bayescap_ckpt,
        div2k_train_loader,
        div2k_val_loader,
        epochs=50,
        init_lr=5e-5,
        device=device,
    )
else:
    print("Skipping BayesCap DIV2K fine-tuning (run_div2k_bayescap=False)")

Epoch 0: 100%|██████████| 100/100 [00:20<00:00,  4.87batch/s, loss=0.268]



Avg. loss: 0.8978620047867298
current score: 0.14035906575561966 | Last best score: -100000000.0
current score: 0.14035906575561966 | Last best score: -100000000.0


Epoch 1: 100%|██████████| 100/100 [00:19<00:00,  5.06batch/s, loss=0.064]



Avg. loss: 0.14754561822861434
current score: 0.2843219699431211 | Last best score: 0.14035906575561966
current score: 0.2843219699431211 | Last best score: 0.14035906575561966


Epoch 2: 100%|██████████| 100/100 [00:24<00:00,  4.06batch/s, loss=0.0238]



...

Avg. loss: -0.07920422993600368
current score: 0.6103375287726521 | Last best score: 0.6217589554935694
current score: 0.6103375287726521 | Last best score: 0.6217589554935694


Epoch 48: 100%|██████████| 100/100 [00:21<00:00,  4.67batch/s, loss=-0.0815]


Avg. loss: -0.07886756937950849
current score: 0.6147329454869032 | Last best score: 0.6217589554935694
current score: 0.6147329454869032 | Last best score: 0.6217589554935694


Epoch 49: 100%|██████████| 100/100 [00:19<00:00,  5.03batch/s, loss=-0.0664]


Avg. loss: -0.08106000002473593
current score: 0.6249246019124984 | Last best score: 0.6217589554935694
current score: 0.6249246019124984 | Last best score: 0.6217589554935694


## 8. Visualize SR outputs & uncertainty
Inspect SR reconstructions, alpha/beta maps, and per-pixel error/uncertainty for a few samples from the validation loader. This provides qualitative intuition that complements the quantitative tables above.

In [ ]:
viz_device = device
viz_dtype = dtype
num_imgs = 0
mean_ssim = 0.0
for idx, batch in enumerate(val_loader):
    print(f'Image {idx} ...')
    xLR, xHR = batch[0].to(viz_device), batch[1].to(viz_device)
    xLR, xHR = xLR.type(viz_dtype), xHR.type(viz_dtype)
    with torch.no_grad():
        xSR = NetG(xLR)
        xSRC_mu, xSRC_alpha, xSRC_beta = NetC(xSR)
    n_batch = xSRC_mu.shape[0]
    for j in range(n_batch):
        num_imgs += 1
        mean_ssim += img_ssim(xSRC_mu[j], xHR[j]).item()

    plt.figure(figsize=(30, 10))
    plt.subplot(1, 4, 1)
    plt.imshow(xLR[0].to('cpu').data.clip(0,1).transpose(0,2).transpose(0,1))
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(xSR[0].to('cpu').data.clip(0,1).transpose(0,2).transpose(0,1))
    plt.axis('off')

    plt.subplot(1, 4, 3)
    a_map = (1/(xSRC_alpha[0] + 1e-5)).to('cpu').data
    plt.imshow(a_map.transpose(0,2).transpose(0,1), cmap='inferno')
    plt.clim(0, 0.1)
    plt.axis('off')

    plt.subplot(1, 4, 4)
    error_map = torch.mean(torch.pow(torch.abs(xSR[0]-xHR[0]),2), dim=0).to('cpu').data
    plt.imshow(error_map, cmap='jet')
    plt.clim(0,0.01)
    plt.axis('off')

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

    plt.figure(figsize=(30,10))
    plt.subplot(1,4,1)
    plt.imshow(xHR[0].to('cpu').data.clip(0,1).transpose(0,2).transpose(0,1))
    plt.axis('off')

    plt.subplot(1,4,2)
    plt.imshow((0.6*xSRC_mu[0]+0.4*xSR[0]).to('cpu').data.clip(0,1).transpose(0,2).transpose(0,1))
    plt.axis('off')

    plt.subplot(1,4,3)
    b_map = xSRC_beta[0].to('cpu').data
    plt.imshow(b_map.transpose(0,2).transpose(0,1), cmap='cividis')
    plt.clim(0.45, 0.75)
    plt.axis('off')

    plt.subplot(1,4,4)
    u_map = (a_map**2)*(torch.exp(torch.lgamma(3/(b_map + 1e-2)))/torch.exp(torch.lgamma(1/(b_map + 1e-2))))
    plt.imshow(u_map.transpose(0,2).transpose(0,1), cmap='hot')
    plt.clim(0,0.15)
    plt.axis('off')

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

print(f'Mean SSIM over {num_imgs} crops: {mean_ssim / max(num_imgs, 1):.4f}')